# Iris Classification with PyTorch and Cloudera AI Inference

This notebook demonstrates a complete machine learning pipeline:
1. **Model Training**: Train an iris classifier using PyTorch with MLflow tracking
2. **Model Export**: Convert PyTorch model to ONNX format for deployment
3. **Batch Inference**: Run inference against deployed models on Cloudera AI Inference

## Prerequisites
- MLflow server running
- Cloudera AI Inference endpoint deployed (for inference section)
- Required Python packages installed

## Part 1: Model Training and Export

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import mlflow.pytorch
import mlflow.onnx
import numpy as np
import pandas as pd
import onnx
import onnxruntime as ort
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

### Define the Neural Network Architecture

In [ ]:
class IrisClassifier(nn.Module):
    def __init__(self, input_size=4, hidden_size=16, num_classes=3):
        super(IrisClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

print("Model architecture defined!")

### Data Preparation

In [ ]:
def prepare_data():
    """Load and prepare the iris dataset"""
    iris = load_iris()
    X, y = iris.data, iris.target
    
    print(f"Dataset shape: {X.shape}")
    print(f"Classes: {iris.target_names}")
    print(f"Features: {iris.feature_names}")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    X_test_tensor = torch.FloatTensor(X_test_scaled)
    y_train_tensor = torch.LongTensor(y_train)
    y_test_tensor = torch.LongTensor(y_test)
    
    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, scaler, iris

# Prepare the data
X_train, X_test, y_train, y_test, scaler, iris_data = prepare_data()

### Visualize the Dataset

In [ ]:
# Create a DataFrame for visualization
iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['target'] = iris_data.target
iris_df['species'] = iris_df['target'].map(dict(enumerate(iris_data.target_names)))

# Create pairplot
plt.figure(figsize=(12, 8))
sns.pairplot(iris_df, hue='species', height=2.5)
plt.suptitle('Iris Dataset - Feature Relationships', y=1.02)
plt.show()

# Class distribution
plt.figure(figsize=(8, 6))
sns.countplot(data=iris_df, x='species')
plt.title('Class Distribution')
plt.show()

### Training Functions

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=100):
    """Train the model and return training history"""
    model.train()
    train_losses = []
    train_accuracies = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        correct = 0
        total = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
        
        avg_loss = epoch_loss / len(train_loader)
        accuracy = 100 * correct / total
        
        train_losses.append(avg_loss)
        train_accuracies.append(accuracy)
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')
    
    return train_losses, train_accuracies

def evaluate_model(model, test_loader):
    """Evaluate the model on test data"""
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())
    
    accuracy = accuracy_score(all_targets, all_predictions)
    return accuracy, all_predictions, all_targets

print("Training functions defined!")

### Model Training with MLflow Tracking

In [ ]:
# Set MLflow experiment
mlflow.set_experiment("iris_classification_notebook")

# Hyperparameters
hidden_size = 16
learning_rate = 0.01
num_epochs = 100
batch_size = 16

print(f"Training configuration:")
print(f"Hidden size: {hidden_size}")
print(f"Learning rate: {learning_rate}")
print(f"Number of epochs: {num_epochs}")
print(f"Batch size: {batch_size}")

In [ ]:
with mlflow.start_run() as run:
    print("Starting MLflow experiment...")
    
    # Log hyperparameters
    mlflow.log_param("hidden_size", hidden_size)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("batch_size", batch_size)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model, loss function, and optimizer
    model = IrisClassifier(input_size=4, hidden_size=hidden_size, num_classes=3)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print("\nTraining model...")
    # Train the model
    train_losses, train_accuracies = train_model(
        model, train_loader, criterion, optimizer, num_epochs
    )
    
    # Evaluate the model
    print("\nEvaluating model...")
    test_accuracy, predictions, targets = evaluate_model(model, test_loader)
    
    # Log metrics
    mlflow.log_metric("final_train_accuracy", train_accuracies[-1])
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("final_train_loss", train_losses[-1])
    
    # Log training curves
    for epoch, (loss, acc) in enumerate(zip(train_losses, train_accuracies)):
        mlflow.log_metric("train_loss", loss, step=epoch)
        mlflow.log_metric("train_accuracy", acc, step=epoch)
    
    print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
    print(f"MLflow Run ID: {run.info.run_id}")

### Visualize Training Progress

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss curve
ax1.plot(train_losses)
ax1.set_title('Training Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True)

# Accuracy curve
ax2.plot(train_accuracies)
ax2.set_title('Training Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.grid(True)

plt.tight_layout()
plt.show()

### Model Evaluation and Classification Report

In [ ]:
# Create and display classification report
class_names = iris_data.target_names
report = classification_report(targets, predictions, target_names=class_names)
print("Classification Report:")
print(report)

# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(targets, predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

### Convert Model to ONNX Format

In [ ]:
def convert_to_onnx(model, input_size=(1, 4), onnx_path="iris_model.onnx"):
    """Convert PyTorch model to ONNX format"""
    model.eval()
    
    # Create dummy input for tracing
    dummy_input = torch.randn(input_size)
    
    # Export to ONNX
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    
    print(f"Model exported to ONNX format: {onnx_path}")
    return onnx_path

def verify_onnx_model(onnx_path, test_data):
    """Verify ONNX model works correctly"""
    # Load ONNX model
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    
    # Create ONNX Runtime session
    ort_session = ort.InferenceSession(onnx_path)
    
    # Test with a small batch
    test_input = test_data[:5].numpy()  # Take first 5 samples
    ort_inputs = {ort_session.get_inputs()[0].name: test_input}
    ort_outputs = ort_session.run(None, ort_inputs)
    
    print(f"ONNX model verification successful. Output shape: {ort_outputs[0].shape}")
    return True

# Convert and verify ONNX model
onnx_path = "iris_model.onnx"
convert_to_onnx(model, input_size=(1, 4), onnx_path=onnx_path)
verify_onnx_model(onnx_path, X_test)

### Log Models to MLflow Registry

In [ ]:
with mlflow.start_run() as run:
    # Log the PyTorch model to MLflow
    print("Logging PyTorch model to MLflow...")
    mlflow.pytorch.log_model(
        model, 
        "iris_classifier_pytorch",
        registered_model_name="iris_pytorch_classifier_notebook"
    )
    
    # Log the ONNX model to MLflow
    print("Logging ONNX model to MLflow...")
    onnx_model = onnx.load(onnx_path)
    mlflow.onnx.log_model(
        onnx_model,
        "iris_classifier_onnx",
        registered_model_name="iris_onnx_classifier_notebook"
    )
    
    print("Models logged successfully to MLflow!")
    print(f"MLflow Run ID: {run.info.run_id}")

## Part 2: Batch Inference (Demo)

**Note**: This section demonstrates the inference client setup. To actually run inference, you need:
1. A deployed model on Cloudera AI Inference
2. Proper authentication credentials
3. Correct endpoint URLs

In [ ]:
# Import inference-related libraries
# Note: Uncomment these imports if you have the required packages installed
# from open_inference.openapi.client import OpenInferenceClient, InferenceRequest
# import httpx
import json
import time
from typing import Optional, Dict, Any, List

print("Inference libraries imported (demo mode)")

### Inference Client Class Definition

In [ ]:
class TritonBatchInference:
    """Class to handle Triton inference with dynamic batching"""
    
    def __init__(self, base_url: str, model_name: str, token: str):
        self.base_url = base_url
        self.model_name = model_name
        self.headers = {
            'Authorization': f'Bearer {token}',
            'Content-Type': 'application/json'
        }
        # Note: Uncomment these lines for actual inference
        # self.httpx_client = httpx.Client(headers=self.headers)
        # self.client = OpenInferenceClient(base_url=base_url, httpx_client=self.httpx_client)
        
    def get_triton_model_config(self) -> Optional[Dict[str, Any]]:
        """Get Triton model configuration including dynamic batching settings"""
        config_url = f"{self.base_url}/v2/models/{self.model_name}/config"
        print(f"Would fetch config from: {config_url}")
        
        # Demo: return mock configuration
        return {
            "preferred_batch_size": [8],
            "max_queue_delay_microseconds": 1000
        }
    
    def check_server_status(self) -> bool:
        """Check if the server is ready and get model metadata"""
        print("Demo: Server status check")
        return True
    
    def prepare_iris_data(self) -> tuple:
        """Load and prepare the iris dataset for inference"""
        # Load iris dataset
        iris = load_iris()
        X, y = iris.data, iris.target
        
        # Split data (we'll use test set for batch inference)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # Scale the features (assuming model was trained with scaled features)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        print(f"Dataset prepared: {len(X_test_scaled)} samples for inference")
        print(f"Feature shape: {X_test_scaled.shape}")
        
        return X_test_scaled, y_test, iris.target_names
    
    def create_batches(self, data: np.ndarray, batch_size: int) -> List[np.ndarray]:
        """Create batches from the input data"""
        batches = []
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            batches.append(batch)
        return batches
    
    def run_batch_inference_demo(self, data: np.ndarray, batch_size: int) -> np.ndarray:
        """Demo version of batch inference"""
        batches = self.create_batches(data, batch_size)
        all_predictions = []
        
        print(f"Demo: Running inference on {len(batches)} batches of size {batch_size}")
        
        for i, batch in enumerate(batches):
            # Simulate inference with random predictions
            batch_predictions = np.random.randint(0, 3, size=len(batch))
            all_predictions.extend(batch_predictions)
            print(f"Demo: Batch {i+1}/{len(batches)} completed")
        
        return np.array(all_predictions)
    
    def evaluate_predictions(self, predictions: np.ndarray, y_true: np.ndarray, 
                           class_names: List[str]) -> Dict[str, Any]:
        """Evaluate the predictions and return metrics"""
        accuracy = accuracy_score(y_true, predictions)
        report = classification_report(y_true, predictions, target_names=class_names)
        
        results = {
            'accuracy': accuracy,
            'classification_report': report,
            'total_samples': len(y_true),
            'correct_predictions': np.sum(predictions == y_true)
        }
        
        return results

print("Inference client class defined!")

### Demo Inference Pipeline

In [ ]:
# Demo configuration (replace with actual values for real inference)
DEMO_BASE_URL = "https://your-inference-endpoint.com"
DEMO_MODEL_NAME = "iris-classifier"
DEMO_TOKEN = "your-auth-token"

# Initialize demo inference client
demo_client = TritonBatchInference(DEMO_BASE_URL, DEMO_MODEL_NAME, DEMO_TOKEN)

print("Demo inference client initialized")
print(f"Base URL: {DEMO_BASE_URL}")
print(f"Model Name: {DEMO_MODEL_NAME}")

In [ ]:
# Run demo inference pipeline
print("=" * 50)
print("DEMO BATCH INFERENCE PIPELINE")
print("=" * 50)

# Check server status (demo)
if demo_client.check_server_status():
    print("✓ Demo server ready")

# Get model configuration (demo)
config = demo_client.get_triton_model_config()
batch_size = config['preferred_batch_size'][0] if config else 8
print(f"Using batch size: {batch_size}")

# Prepare data
X_inference, y_inference, class_names = demo_client.prepare_iris_data()

# Run demo inference
print("\nRunning demo batch inference...")
start_time = time.time()
demo_predictions = demo_client.run_batch_inference_demo(X_inference, batch_size)
total_time = time.time() - start_time

# Evaluate demo results
results = demo_client.evaluate_predictions(demo_predictions, y_inference, class_names)
results['total_inference_time'] = total_time
results['avg_time_per_sample'] = total_time / len(X_inference)
results['throughput_samples_per_second'] = len(X_inference) / total_time

print("\n" + "=" * 50)
print("DEMO INFERENCE RESULTS")
print("=" * 50)
print(f"Total Samples: {results['total_samples']}")
print(f"Correct Predictions: {results['correct_predictions']}")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Total Inference Time: {results['total_inference_time']:.3f}s")
print(f"Average Time per Sample: {results['avg_time_per_sample']:.6f}s")
print(f"Throughput: {results['throughput_samples_per_second']:.2f} samples/second")

print("\nDemo Classification Report:")
print(results['classification_report'])

### Real Inference Setup Instructions

To run actual inference against a deployed model:

1. **Deploy your ONNX model** to Cloudera AI Inference
2. **Update configuration variables**:
   ```python
   BASE_URL = 'https://your-actual-endpoint.com'
   MODEL_NAME = 'your-actual-model-name'
   ```
3. **Set up authentication**:
   ```python
   # Option 1: Token file
   def load_token(token_path="/tmp/jwt"):
       with open(token_path, 'r') as f:
           token_data = json.load(f)
           return token_data["access_token"]
   
   # Option 2: Environment variable
   import os
   token = os.getenv('CDP_TOKEN')
   ```
4. **Install required packages**:
   ```bash
   pip install open-inference httpx
   ```
5. **Replace demo client with real client**:
   ```python
   # Uncomment the httpx and OpenInferenceClient imports
   # Initialize real client instead of demo
   ```

## Summary

This notebook demonstrated:

1. **Complete ML Pipeline**: From data loading to model deployment
2. **PyTorch Model Training**: Neural network with proper validation
3. **MLflow Integration**: Experiment tracking and model registry
4. **ONNX Conversion**: Model format conversion for deployment
5. **Inference Client**: Framework for batch inference against deployed models

### Next Steps:
- Deploy the ONNX model to Cloudera AI Inference
- Configure real authentication and endpoint URLs
- Run production batch inference
- Monitor model performance and retrain as needed

In [ ]:
# Clean up temporary files
import os

files_to_clean = ["iris_model.onnx"]
for file_path in files_to_clean:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Cleaned up: {file_path}")

print("Notebook execution completed!")